In [3]:
import cv2
import numpy as np
from tkinter import *
from tkinter import filedialog
from PIL import Image, ImageTk

In [5]:
def apply_filter(filter_type):
    global img, panel
    if img is None:
        return

    img_copy = img.copy()

    if filter_type == "Blur":
        result = cv2.GaussianBlur(img_copy, (15, 15), 0)

    
    elif filter_type == "Sharpen":
        kernel = np.array([[0, -1, 0],
                           [-1, 5,-1],
                           [0, -1, 0]])
        result = cv2.filter2D(img_copy, -1, kernel)
   
    
    elif filter_type == "Sobel":
        gray = cv2.cvtColor(img_copy, cv2.COLOR_BGR2GRAY)
        sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=5)
        sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=5)
        result = cv2.magnitude(sobelx, sobely)
        result = cv2.normalize(result, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
        result = cv2.cvtColor(result, cv2.COLOR_GRAY2BGR)
    
    
    elif filter_type == "Canny":
        gray = cv2.cvtColor(img_copy, cv2.COLOR_BGR2GRAY)
        result = cv2.Canny(gray, 100, 200)
        result = cv2.cvtColor(result, cv2.COLOR_GRAY2BGR)
    
    
    elif filter_type == "Affine":
        rows, cols, ch = img_copy.shape
        pts1 = np.float32([[50, 50], [200, 50], [50, 200]])
        pts2 = np.float32([[10, 100], [200, 50], [100, 250]])
        M = cv2.getAffineTransform(pts1, pts2)
        result = cv2.warpAffine(img_copy, M, (cols, rows))
    
    
    else:
        result = img_copy

    
    
    # Convert image for Tkinter
    result_rgb = cv2.cvtColor(result, cv2.COLOR_BGR2RGB)
    im = Image.fromarray(result_rgb)
    imgtk = ImageTk.PhotoImage(image=im)

    panel.configure(image=imgtk)
    panel.image = imgtk





In [7]:
def load_image():
    global img, panel
    file_path = filedialog.askopenfilename()
    if not file_path:
        return
    img = cv2.imread(file_path)

    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    im = Image.fromarray(img_rgb)
    imgtk = ImageTk.PhotoImage(image=im)

    panel.configure(image=imgtk)
    panel.image = imgtk




In [9]:
root = Tk()
root.title("Image Filters & Transformations")

img = None

btn_frame = Frame(root)
btn_frame.pack()

btn_load = Button(btn_frame, text="Load Image", command=load_image)
btn_load.grid(row=0, column=0)

filters = ["Blur", "Sharpen", "Sobel", "Canny", "Affine"]

for idx, f in enumerate(filters):
    btn = Button(btn_frame, text=f, command=lambda f=f: apply_filter(f))
    btn.grid(row=0, column=idx+1)

panel = Label(root)
panel.pack()

root.mainloop()